In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from SimLib import config_sim as conf
from SimLib import sipm_mapping as DAQ
import sys
sys.path.append("/home/viherbos/GITHUB/PETALO_analysis")
import fit_library
import scipy.signal as sc

%matplotlib nbagg

In [16]:
class TOF_compute(object):
    """ SIPM : [risetime_tau (ps), falltime_tau (ps)]
    """
    def __init__(self, path, file_name, SIPM, time_bin=5):
        self.p_name = path
        self.f_name = file_name
        self.time_bin = time_bin
        
        # SPE response computation
        self.n_sipms    = SIPM['n_sipms']
        self.first_sipm = SIPM['first_sipm']
        
        tau_sipm   = SIPM['tau_sipm']        
        time     = np.arange(0,80000,time_bin)
        alfa = 1.0/tau_sipm[1]
        beta = 1.0/tau_sipm[0]
        t_p = np.log(beta/alfa)/(beta-alfa)
        K = (beta)*np.exp(alfa*t_p)/(beta-alfa)
        self.spe_resp = K*(np.exp(-time*alfa)-np.exp(-time*beta))
        
        os.chdir(path)
        self.tof_wave   = np.array(pd.read_hdf(file_name,key='MC/tof_waveforms'), 
                                   dtype='int')       
        
    def __call__(self,event, time_window, TDC_TE=1):
        event_select    = np.argwhere(self.tof_wave[:,0]==event)
        event_tof       = self.tof_wave[event_select[:,0],1:]            
        event_tof[:,0]  = event_tof[:,0]*-1-self.first_sipm
        # SiPM  |  Timebin  |  Charge

        # Beware of Empty Events
        time_length = np.max(event_tof[:,1])
        #print ("Problema: %d" % time_length)
        pe_table = np.zeros((time_length+1,self.n_sipms))

        for i in range(event_tof.shape[0]):
            pe_table[event_tof[i,1],event_tof[i,0]] = event_tof[i,2]

        if time_window == -1: 
            conv_table = np.zeros((np.max(event_tof[:,1])+1 + self.spe_resp.shape[0]-1,
                                   self.n_sipms))
        else:
            pe_table = pe_table[0:time_window,:]
            conv_table = np.zeros((pe_table.shape[0] + self.spe_resp.shape[0]-1,
                                   self.n_sipms))

        for i in range(self.n_sipms):
            if np.max(pe_table[:,i])>0:
                conv_table[:,i] = np.convolve(pe_table[:,i],self.spe_resp)
                #conv_table[:,i] = sc.fftconvolve(pe_table[:,i],self.spe_resp)

        charge_acc = np.cumsum(conv_table,axis=0)
        #part_acc   = np.cumsum(pe_table,axis=0)

        timestamp_v = np.array([])
        for i in range(conv_table.shape[1]):
            timestamp  = np.argwhere(conv_table[0:time_window,i]>TDC_TE)
            # We take only the first part up to time_window to speed up the computation
            if timestamp.size == 0:
                timestamp = 0
            else:
                timestamp  = np.min(timestamp)
                
            timestamp_v = np.hstack([timestamp_v,timestamp])

        return timestamp_v, charge_acc
    
    


# Extract Event Information

In [17]:
SIPM = {'n_sipms':3500, 'first_sipm':1000, 'tau_sipm':[250,15000]}
TDC = TOF_compute("/mnt/715c6d30-57c4-4aed-a982-551291d8f848/NEUTRINOS/","petit_ring_tof.h5",
                                 SIPM       = SIPM,
                                 time_bin   = 5)

# Configuration Reading

In [18]:
path         = "/home/viherbos/DAQ_DATA/NEUTRINOS/PETit-ring/7mm_pitch/"
jsonfilename = "CUBE"
SIM_CONT=conf.SIM_DATA(filename=path+jsonfilename+".json",read=True)
data = SIM_CONT.data
L1_Slice, Matrix_I, Matrix_O, topo = DAQ.SiPM_Mapping(data,data['L1']['map_style'])


Number of SiPM : 3500 
Number of ASICS : 55 
Minimum Number of L1 : 6 
Available ASICS = 55
Connected ASICS = 55
Instanciated L1 = 6
L1 number 0 has 9 ASICs
L1 number 1 has 9 ASICs
L1 number 2 has 9 ASICs
L1 number 3 has 10 ASICs
L1 number 4 has 9 ASICs
L1 number 5 has 9 ASICs


# Image Composing

In [5]:
timestamp, accu_table = TDC(100,-1,1)   # Event, Time Window for TDC, TE_TDC
print np.min(timestamp[np.argwhere(timestamp>0)])

238.0


In [6]:
time = 20000
first_SiPM = 1000
phi_o = np.zeros(Matrix_O.shape)
# Data Matrix Composition
for i in range(Matrix_O.shape[0]):
    for j in range(Matrix_O.shape[1]):
        phi_o[i,j] = accu_table[time,int(Matrix_O[i,j])]

In [7]:
fig = plt.figure(figsize=(10,1.5))
plt.imshow(phi_o)
plt.show()
print np.min(timestamp[np.argwhere(timestamp>0)])*5

<IPython.core.display.Javascript object>

1190.0


# TOF computation

In [19]:
def TOF_comp(time_window, TE_TDC):
    t_stamp_v = np.array([]).reshape(0,3500)
    ring_dim     = Matrix_O.shape
    TOF = np.array([])
    j=0

    for i in range(10,3556):
        timestamp, charge_acc = TDC(i,400,1)
        print ("Process Event : %d" % i)
        t_stamp_v = np.vstack([t_stamp_v,timestamp])

        timestamp_M    = np.ma.MaskedArray(timestamp,timestamp<1)
        gamma1_sipm    = np.ma.argmin(timestamp_M)
        gamma1_tdc     = np.ma.min(timestamp_M)
        gamma2_sipm    = np.zeros(gamma1_sipm.shape)
        gamma2_tdc     = np.zeros(gamma1_sipm.shape)

        gamma1_coord = np.where(Matrix_O==gamma1_sipm)
        print ("Gamma 1 SiPM : %d" % Matrix_O[gamma1_coord])

        # Roll SiPM Matrixes to find opposite side of detector
        Xe = np.roll(Matrix_O,-gamma1_coord[1]+ring_dim[1]//4,axis=1)
        # Select opposite side of detector
        Xe_sel = Xe[:,ring_dim[1]//2:]
        Xe_sel_1D = Xe_sel.reshape(-1)
        try:
            OPO_g = timestamp_M[Xe_sel_1D]
            gamma2_tdc = np.ma.min(OPO_g)
            gamma2_coord = Xe_sel_1D[np.ma.argmin(OPO_g)]
            print ("Gamma 2 SiPM : %d" % gamma2_coord)
        except:
            gamma2_tdc = 0

        # Get rid of singles
        TOF_p = (gamma1_tdc - gamma2_tdc)/2
        if np.logical_not(np.isnan(TOF_p)):
            TOF = np.append(TOF,TOF_p)
            print gamma1_tdc, gamma2_tdc, TOF_p
            j += 1
        
        # Introduce a random sign to symmetrize distribution
        random_sign = (np.random.rand(len(TOF))>0.5)*-1
        random_sign = random_sign + random_sign + 1
        TOF = TOF * random_sign  
    
    return TOF
    

In [20]:
TOF_1 = TOF_comp(300,1)

Process Event : 10
Gamma 1 SiPM : 469
Gamma 2 SiPM : 163
Process Event : 11
Gamma 1 SiPM : 2690
Gamma 2 SiPM : 109
Process Event : 12
Gamma 1 SiPM : 3261
Gamma 2 SiPM : 155
Process Event : 13
Gamma 1 SiPM : 0
Gamma 2 SiPM : 44
Process Event : 14
Gamma 1 SiPM : 2716
Gamma 2 SiPM : 135
Process Event : 15
Gamma 1 SiPM : 3460
Gamma 2 SiPM : 4
Process Event : 16
Gamma 1 SiPM : 3065
Gamma 2 SiPM : 134
Process Event : 17
Gamma 1 SiPM : 3084
Gamma 2 SiPM : 153
Process Event : 18
Gamma 1 SiPM : 2024
Gamma 2 SiPM : 143
Process Event : 19
Gamma 1 SiPM : 2437
Gamma 2 SiPM : 1306
199.0 319.0 -60.0
Process Event : 20
Gamma 1 SiPM : 227
Gamma 2 SiPM : 96
Process Event : 21
Gamma 1 SiPM : 1122
Gamma 2 SiPM : 116
Process Event : 22
Gamma 1 SiPM : 2657
Gamma 2 SiPM : 76
Process Event : 23
Gamma 1 SiPM : 2580
Gamma 2 SiPM : 174
Process Event : 24
Gamma 1 SiPM : 2198
Gamma 2 SiPM : 142
Process Event : 25
Gamma 1 SiPM : 2449
Gamma 2 SiPM : 43
Process Event : 26
Gamma 1 SiPM : 1554
Gamma 2 SiPM : 23
Process

KeyboardInterrupt: 

In [ ]:
TOF_2 = TOF_comp(300,2)

In [ ]:
TOF_3 = TOF_comp(300,3)

In [ ]:
TOF_4 = TOF_comp(300,4)

In [ ]:
TOF_5 = TOF_comp(300,5)

In [ ]:
TOF_6 = TOF_comp(300,6)

In [ ]:
fit = fit_library.gauss_fit()
fig3 = plt.figure(figsize=(8,6))
data = TOF_1[(TOF_1>-400)*(TOF_1<400)]*5
fit(TOF_1,50)
fit.plot(axis = fig3.add_subplot(321),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
data = TOF_2[(TOF_2>-400)*(TOF_2<400)]*5
fit(TOF_2,50)
fit.plot(axis = fig3.add_subplot(322),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
data = TOF_3[(TOF_3>-400)*(TOF_3<400)]*5
fit(TOF_3,50)
fit.plot(axis = fig3.add_subplot(323),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
data = TOF_4[(TOF_4>-400)*(TOF_4<400)]*5
fit(TOF_4,50)
fit.plot(axis = fig3.add_subplot(324),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
data = TOF_5[(TOF_5>-400)*(TOF_5<400)]*5
fit(TOF_5,50)
fit.plot(axis = fig3.add_subplot(325),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
data = TOF_6[(TOF_6>-400)*(TOF_6<400)]*5
fit(TOF_6,50)
fit.plot(axis = fig3.add_subplot(326),
         title = "Time of Flight ",
         xlabel = "Time Stamp in picoseconds",
         ylabel = "Hits",
         res=False, 
         fit=True)
fig3.tight_layout()

In [ ]:
TOF